In [1]:
%reload_ext autoreload
%autoreload 2

# Logistic Regression Models

In [2]:
import autorootcwd
import json
import matplotlib.pyplot as plt
import os
import pandas as pd
from tqdm import tqdm

### Performances

In [33]:
rows = []
DIR = "./results/logistic_lr_search"

for lr_scheduler in ['cosine', 'plateau', 'none']:
    for lr in [0.0001, 0.001, 0.01]:
        with open(f"{DIR}/lr_{lr}_scheduler_{lr_scheduler}/val_metrics.json") as f:
            data = json.load(f)
            rows.append({
                'lr_scheduler': lr_scheduler,
                'lr': lr,
                'acc': data['mean']['acc'],
                'root': data['mean']['root'],
                'third': data['mean']['third'],
                'seventh': data['mean']['seventh'],
                'mirex': data['mean']['mirex'],
                'class_wise_mean': data['class_wise']['acc']['mean'],
                'class_wise_median': data['class_wise']['acc']['median'],
            })

df = pd.DataFrame(rows)

In [34]:
df

,lr_scheduler,lr,acc,root,third,seventh,mirex,class_wise_mean,class_wise_median
0,cosine,0.0001,27.948654,49.489602,39.154729,29.054274,46.042144,6.328708,0.269696
1,cosine,0.0010,40.780169,62.206331,54.334969,42.382093,61.657118,11.455583,2.325280
2,cosine,0.0100,42.684485,63.972367,56.523239,44.299777,64.602473,12.705191,2.189911
3,plateau,0.0001,32.134865,54.310574,44.698521,33.435841,52.125744,8.084931,0.935640
4,plateau,0.0010,42.981399,64.464031,56.901664,44.665704,60.883659,12.024866,1.709139
5,plateau,0.0100,41.999672,63.861619,56.750138,43.601327,62.118015,12.947601,1.257826
6,none,0.0001,32.134865,54.310574,44.698521,33.435841,52.125744,8.084931,0.935640
7,none,0.0010,41.408788,63.273262,56.008067,43.004530,61.328560,12.280139,0.804367
8,none,0.0100,28.754232,51.444754,41.198715,29.867983,50.989947,8.972872,0.203339


### Training Graphs

In [ ]:
# Loop over directory of experiments
DIR = './results/logistic_lr_search'
experiments = os.listdir(DIR)
for lr_scheduler in ['cosine', 'plateau', 'none']:
    for lr in [0.0001, 0.001, 0.01]:
        if lr_scheduler == 'none' and lr == 0.0001:
            continue
        exp = f'lr_{lr}_scheduler_{lr_scheduler}'
        training_history = json.load(open(f'{DIR}/{exp}/training_history.json'))
        train_losses = training_history['train_losses']
        val_losses = training_history['val_losses']
        val_accuracies = training_history['val_accuracies']
        # Plot all 3, with acc having its own y-axis
        fig, ax1 = plt.subplots()
        ax2 = ax1.twinx()
        ax1.plot(train_losses, label='Training Loss', color='blue')
        # Multiply validations by 5 as they were only recorded every 5 epochs
        ax1.plot([5 * i for i in range(len(val_losses))], val_losses, label='Validation Loss', color='red')
        ax2.plot([5 * i for i in range(len(val_accuracies))], val_accuracies, label='Validation Accuracy', color='green')
        ax1.set_xlabel('Epoch')
        ax1.set_ylabel('Loss')
        ax2.set_ylabel('Accuracy')
        ax1.legend(loc='upper left')
        plt.title(f'{exp}')

# CNNs

In [31]:
rows = []
DIR = "./results/cnns"

ks = [5, 5, 9]
ls = [1,3,5]
cs = [1,5,10]
for k, l, c in zip(ks, ls, cs):
    with open(f"{DIR}/cnn_k{k}_l{l}_c{c}/val_metrics.json") as f:
        data = json.load(f)
    rows.append({
        'k': k,
        'l': l,
        'c': c,
        'acc': data['mean']['acc'],
        'root': data['mean']['root'],
        'third': data['mean']['third'],
        'seventh': data['mean']['seventh'],
        'mirex': data['mean']['mirex'],
        'class_wise_mean': data['class_wise']['acc']['mean'],
        'class_wise_median': data['class_wise']['acc']['median'],
    })

df = pd.DataFrame(rows)

In [32]:
df

,k,l,c,acc,root,third,seventh,mirex,class_wise_mean,class_wise_median
0,5,1,1,54.524142,74.350484,69.021008,56.645015,73.481676,16.041350,2.261545
1,5,3,5,56.972763,76.899774,72.496475,59.164435,77.577848,18.903405,3.035360
2,9,5,10,57.784539,78.139240,73.960484,60.009495,77.758385,19.248573,3.178795


## CR2

In [37]:
rows = []
DIR = "./results/cr2"

for cr2 in ['-on', '_off']:
    with open(f"{DIR}/cr2{cr2}/val_metrics.json") as f:
        data = json.load(f)
    rows.append({
        'cr2': 'on' if cr2 == '-on' else 'off',
        'acc': data['mean']['acc'],
        'root': data['mean']['root'],
        'third': data['mean']['third'],
        'seventh': data['mean']['seventh'],
        'mirex': data['mean']['mirex'],
        'class_wise_mean': data['class_wise']['acc']['mean'],
        'class_wise_median': data['class_wise']['acc']['median'],
    })

df = pd.DataFrame(rows)

In [38]:
df

,cr2,acc,root,third,seventh,mirex,class_wise_mean,class_wise_median
0,on,59.703930,78.952111,75.611299,61.974481,80.478331,18.433995,0.379817
1,off,60.198375,78.428361,75.338128,62.468808,79.479692,19.376959,1.083215
